# Building an ETL Pipeline

In [1]:
import pandas as pd
#import mysql.connector 
import pymysql
from sqlalchemy import create_engine
from datetime import datetime, date, time, timedelta
import tweepy
import numpy as np 
import pyodbc
from sqlalchemy import create_engine
import urllib
import re

### Connecting to Twitter

In [2]:
# Consumer:
CONSUMER_KEY    = 'uXfKji3NyWLDEGI3gXxx3tBNc'
CONSUMER_SECRET = 'eiuC2xhRfbmbY2gSplZOcL4dPSSUiqmB8SbLk6eGuZ2DaBaYQj'

# Access:
ACCESS_TOKEN  = '1219607153163603968-oOHm1E2uXjyBiBOkkKNExfAwvU4dVd'
ACCESS_SECRET = '3dZ7jyldTV1m1RxoVPv8VEOT92m2OwEPpF1Syj0j2Mu95'

In [3]:
# # Consumer:
# CONSUMER_KEY    = 'egoItCf7bZMp4gopTCXC0WH2O'
# CONSUMER_SECRET = 'Ph7fyNWzMnLTb0rgx5w0GPb0bzWlfyUtfVhZZGt5jVk4FJeBxK'
# # Access:
# ACCESS_TOKEN  = '840149748354965504-6fGpkvdj6n53uVG5231Oq6PhyLzHlfO'
# ACCESS_SECRET = 'f17t2HIfmpsgh1IBxgdugigEH8Xuzhps7gjGT2jfLOgxT'

In [4]:
# API's setup:
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with access and consumer keys from Twitter.
    """

    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication:
    api = tweepy.API(auth, timeout=1000)
    return api

In [5]:
def twitter_df():
    api = twitter_setup()
    all_tweets = []
    tweets = api.user_timeline(screen_name='ESKOM_SA', 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    all_tweets.extend(tweets)
  
    outtweets = [[tweet.full_text.encode("utf-8").decode("utf-8"),tweet.created_at] 
              for idx,tweet in enumerate(all_tweets)]
    df = pd.DataFrame(outtweets,columns=['Tweets','Date'])
    df.set_index('Tweets')
    return df
twitter_df()

,Tweets,Date
0,"@SibiyaFezeka Yes, the hours remains the same....",2020-03-09 12:41:36
1,"@fatabulaz @City_Ekurhuleni Pls, call Customer...",2020-03-09 12:40:15
2,@luvhengoJ Please contact Customer Services on...,2020-03-09 12:39:35
3,"To register a meter, visit an Eskom office wit...",2020-03-09 12:30:00
4,@LetlatsaS Please call customer services on 08...,2020-03-09 10:08:59
5,@unsauvage Please check with your local munici...,2020-03-09 10:07:22
6,Prepaid pricing is based on usage/ consumption...,2020-03-09 08:15:00
7,@ReardonSandra Query escalated.,2020-03-09 08:03:40
8,@lis_pendes This depends on whether you are su...,2020-03-09 07:37:20
9,@007MrBondZA Will communicate it as soon as re...,2020-03-09 07:33:12


In [6]:
municipality_dict = { '@CityofCTAlerts' : 'Cape Town',
            '@CityPowerJhb' : 'Johannesburg',
            '@eThekwiniM' : 'eThekwini' ,
            '@EMMInfo' : 'Ekurhuleni',
            '@centlecutility' : 'Mangaung',
            '@NMBmunicipality' : 'Nelson Mandela Bay',
            '@CityTshwane' : 'Tshwane'}
def extract_municipality_hashtags(df):

    mun_dict = {'@CityofCTAlerts' : 'Cape Town',
                '@CityPowerJhb' : 'Johannesburg',
                '@eThekwiniM' : 'eThekwini' ,
                '@EMMInfo' : 'Ekurhuleni',
                '@centlecutility' : 'Mangaung',
                '@NMBmunicipality' : 'Nelson Mandela Bay',
                '@CityTshwane' : 'Tshwane'}
    
    handles = list(mun_dict.keys())
    
    df['municipality'] = df['Tweets'].str.extract('({})'.format('|'.join(handles)), expand=False).fillna(np.nan)
    df['municipality'] = df['municipality'].map(mun_dict)
    
    df['hashtags'] = df['Tweets'].str.findall(r'#.*?(?=\s|$)')
    df['hashtags'] = df['hashtags'].apply(lambda x: np.nan if len(x)==0 else [x.lower() for x in x])
    
    return df
extract_municipality_hashtags(twitter_df())

,Tweets,Date,municipality,hashtags
0,"@SibiyaFezeka Yes, the hours remains the same....",2020-03-09 12:41:36,NaN,NaN
1,"@fatabulaz @City_Ekurhuleni Pls, call Customer...",2020-03-09 12:40:15,NaN,NaN
2,@luvhengoJ Please contact Customer Services on...,2020-03-09 12:39:35,NaN,NaN
3,"To register a meter, visit an Eskom office wit...",2020-03-09 12:30:00,NaN,NaN
4,@LetlatsaS Please call customer services on 08...,2020-03-09 10:08:59,NaN,NaN
5,@unsauvage Please check with your local munici...,2020-03-09 10:07:22,NaN,NaN
6,Prepaid pricing is based on usage/ consumption...,2020-03-09 08:15:00,NaN,NaN
7,@ReardonSandra Query escalated.,2020-03-09 08:03:40,NaN,NaN
8,@lis_pendes This depends on whether you are su...,2020-03-09 07:37:20,NaN,NaN
9,@007MrBondZA Will communicate it as soon as re...,2020-03-09 07:33:12,NaN,NaN


In [7]:
df = extract_municipality_hashtags(twitter_df())

### Connecting Data Base

In [8]:
df['hashtags'] =(df['hashtags']).astype(str)

In [9]:
conn = pyodbc.connect(driver='{SQL Server}' ,
                     host='MANDLA63\SQLEXPRESS' ,
                     database='gather_eskom' ,
                     trusted_connection='true',
                      user='sa')
twitter_table= pd.read_sql_query('select * from dbo.twitter_table',conn)

In [10]:
# host = COMPUTERNAME\SQLEXPRESS
#creating twitter_table

def pyodbc_twitter(connection, df, twitter_table):
    params = urllib.parse.quote_plus("DRIVER={SQL Server};SERVER=MANDLA63\SQLEXPRESS;DATABASE=gather_eskom;UID=sa;trusted_connection='true'")
    engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

    #df = df2
    df.to_sql('twitter_table', con=engine, if_exists='replace')
    return None
pyodbc_twitter(conn, df, twitter_table)

In [11]:
# Check that twitter table is in your MS SQL Database
df = pd.read_sql_query('select * from dbo.twitter_table',conn)
df.head()

,index,Tweets,Date,municipality,hashtags
0,0,"@SibiyaFezeka Yes, the hours remains the same....",2020-03-09 12:41:36,None,nan
1,1,"@fatabulaz @City_Ekurhuleni Pls, call Customer...",2020-03-09 12:40:15,None,nan
2,2,@luvhengoJ Please contact Customer Services on...,2020-03-09 12:39:35,None,nan
3,3,"To register a meter, visit an Eskom office wit...",2020-03-09 12:30:00,None,nan
4,4,@LetlatsaS Please call customer services on 08...,2020-03-09 10:08:59,None,nan
